In [1]:
import itertools

L=[[-1,-1,0,0],[1,-1,0,0],[0,1,-1,0],[0,0,1,-1]] #D4

n=len(L)

def lincomb_lat(c):
    return [sum(c[k]*L[k][j] for k in range(n)) for j in range(n)]

###### voronoi cell
target_volume = abs(matrix(L).determinant())
print("target volume:",target_volume)

K = 0
vol = target_volume+1
while vol>target_volume: 
    K += 1
    ineqs = []
    for coefs in itertools.product(range(-K,K+1),repeat=n):
        #0\le \|p\|^2-\langle x, 2p\rangle, where p is the lattice point; 0 doesn't need to be excluded as 0\le 0 always
        p = lincomb_lat(coefs)
        ineqs.append([sum((p[i])^2 for i in range(n))]+[-2*p[i] for i in range(n)])
    V = Polyhedron(ieqs=ineqs)
    vol = V.volume() 

   
r = V.radius()
print("radius:",r,r.n())

#need lattice points which are at most 4r from the origin
M = ceil(4*r)
r2 = 16*r^2

#by simple estimate \|G^{-1}y\|_\infty\le \|G^{-1}\|_1 \|y\|_\infty, where \|G^{-1}\|_1=:G1 is the max l1 norm of row vectors in matrix G^{-1}
#so it suffices to check x with \|x\|_\infty\le 4r/G1
GI = matrix(L).inverse()
G1 = max(sqrt(sum((GI[j][i])^2 for j in range(n))) for i in range(n))
M = floor(4*r*G1)
nodes = [] 
for coefs in itertools.product(range(-M,M+1),repeat=n):
    node = [sum(coefs[k]*L[k][j] for k in range(n)) for j in range(n)]
    norm2 = sum((node[i])^2 for i in range(n))
    if norm2 <= r2 and norm2>0:
        nodes.append([vector(node)/2,node.copy()])
print("nodes:",len(nodes))

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(n)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in range(len(nodes)):
            if p not in forb:
                proj_pnt = proj[0]*(nodes[p][0]-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (nodes[p][0]-proj_pnt).norm()
                    if dist<r:
                        forb.append(p)

forbd=[nodes[i][1] for i in forb]

print("forbidden:",len(forbd))


def find_smallest_mod(cc):
    A = set()
    maxv = 0
    for pt in forbd:
        p = abs(sum(cc[i]*pt[i] for i in range(n)))
        if p==0:
            return Infinity
        if p>maxv:
            maxv=p
        A.add(p)
    for i in range(1,maxv+2):
        found = True
        for k in range(1,floor(maxv/i)+1):
            if i*k in A:
                found = False
        if found:
            return i

        
#exhaustive verification up to K
K = 56
minv = Infinity
SS = set([i+1 for i in range(K)])
order = [(-1)^i*(1/4+i/2)-1/4 for i in range(K)]
C = []
for ccc in itertools.product(order,repeat=n):
    if ccc[0]>=0:
        C.append(list(ccc))
print(len(C))
for c in C:
    found = True
    S = SS.copy()
    for pt in forbd:
        p = abs(sum(c[i]*pt[i] for i in range(n)))
        if p==0:
            found = False
            break
        S -= {p}
        if not S:
            found = False
            break
    if found:
        v = find_smallest_mod(c)
        if v<minv:
            print(c,v)
            minv = v
        
print("done")        


target volume: 2
radius: 1 1.00000000000000
nodes: 624
forbidden: 408
4917248
[1, -9, -14, -25] 56
done
